In [4]:
import pandas as pd
import requests
import json
from pprint import pprint
from config import openWeatherKey

ModuleNotFoundError: No module named 'config'

In [5]:
#Pulling data from csv files and creating dataframe with disaster data
coordinates_csv = "output_data/NEW_coordinates.csv"
weatherDisasterCSV = "Resources/disaster_data.csv"
wDisaster_df = pd.read_csv(weatherDisasterCSV)
desiredColumns = ["Dis No", "Year", "Seq", "Disaster Subgroup", "Disaster Type", "Disaster Subtype", "Disaster Subsubtype", "Entry Criteria", "ISO", "Location", "Start Month", "End Month", "Total Deaths", "No Injured", "No Affected", "No Homeless", "Total Affected", "Insured Damages ('000 US$)", "Total Damages ('000 US$)", "CPI"]
wDisaster_df = wDisaster_df.loc[wDisaster_df["ISO"] == "USA", desiredColumns]
wDisaster_df

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Entry Criteria,ISO,Location,Start Month,End Month,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI
2,2000-0080-USA,2000,80,Hydrological,Flood,Riverine flood,NaN,Affect,USA,"Kentucky, Ohio provinces",2,2.0,3.0,NaN,231.0,NaN,231.0,NaN,NaN,67.355759
3,2000-0500-USA,2000,500,Hydrological,Flood,Riverine flood,NaN,Affect,USA,"Morris, Sussex districts (New Jersey province)",8,8.0,NaN,NaN,175.0,NaN,175.0,NaN,166000.0,67.355759
4,2000-0919-USA,2000,919,Meteorological,Storm,NaN,NaN,SigDam,USA,"Alabama, Georgia, Louisiana, North Carolina, S...",1,1.0,4.0,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
5,2000-9712-USA,2000,9712,Climatological,Drought,Drought,NaN,Affected,USA,Wyoming province,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.355759
6,2000-0838-USA,2000,838,Meteorological,Storm,Convective storm,Winter storm/Blizzard,Kill,USA,"Texas, Oklahoma, New Mexico, Arkansas, Missour...",12,12.0,57.0,34.0,NaN,NaN,34.0,200500.0,10000.0,67.355759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,2019-0307-USA,2019,307,Hydrological,Flood,NaN,NaN,Affected,USA,"Lincolnton region (Lincoln County, North Carol...",6,6.0,3.0,NaN,600.0,NaN,600.0,NaN,NaN,100.000000
533,2020-0148-USA,2020,148,Meteorological,Storm,Convective storm,Tornado,Kill,USA,"Louisiana, Texas, Mississippi, South Carolina,...",4,4.0,38.0,200.0,NaN,NaN,200.0,NaN,1000000.0,NaN
534,2020-0011-USA,2020,11,Meteorological,Storm,Convective storm,Severe storm,Kill,USA,"Texas, Oklahoma, Missouri, Arkansas, Louisiana...",1,1.0,12.0,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN
535,2020-0082-USA,2020,82,Meteorological,Storm,Convective storm,Tornado,Waiting,USA,"Nashville (Tennessee), Kentucky, Missouri, Mis...",3,3.0,25.0,300.0,12000.0,NaN,12300.0,NaN,1100000.0,NaN


In [6]:
#Breaksdown and counts all unique locations
wDisaster_df["Location"].value_counts()

Texas province                                                                                                                     4
California province                                                                                                                3
Illinois, Indiana, Iowa, Kansas, Michigan, Minnesota, Missouri, Nebraska, North Dakota, Ohio, South Dakota, Wisconsin provinces    3
Los Angeles district (California province)                                                                                         3
Arizona province                                                                                                                   3
                                                                                                                                  ..
Franklin, Jefferson, Gasconade districts (Missouri province)                                                                       1
Pike district (Kentucky province), Virginia province                 

In [7]:
#Creating new dataframe by splitting data into Location and Dis. No.
newData = {
    "Location": [],
    "Dis No": []
}

for index, row in wDisaster_df.iterrows():
    #print(type(newData["Location"]))
    if type(row["Location"]) is str:
        for location in row["Location"].split(","):
            newData["Location"].append(location)
            newData["Dis No"].append(row["Dis No"])
splitLocations_df = pd.DataFrame(newData)
#splitLocations_df

In [8]:
#Grouping disaster dataframe and counting based on disaster type
wDisaster_df.groupby("Disaster Type").count()

,Dis No,Year,Seq,Disaster Subgroup,Disaster Subtype,Disaster Subsubtype,Entry Criteria,ISO,Location,Start Month,End Month,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI
Disaster Type,,,,,,,,,,,,,,,,,,,
Drought,11,11,11,11,11,0,11,11,11,11,8,0,0,0,0,0,3,9,11
Earthquake,9,9,9,9,9,0,9,9,9,9,9,3,7,7,2,9,4,8,9
Epidemic,3,3,3,3,2,0,3,3,2,3,3,1,0,3,0,3,0,0,3
Extreme temperature,13,13,13,13,13,2,13,13,13,13,13,12,1,0,0,1,3,3,13
Flood,94,94,94,94,79,0,93,94,94,94,94,70,7,69,5,73,22,54,94
Landslide,3,3,3,3,3,0,3,3,3,3,3,3,2,1,1,3,0,2,3
Storm,278,278,278,278,258,203,278,278,278,278,278,238,102,86,29,155,175,224,274
Volcanic activity,1,1,1,1,1,0,1,1,1,1,1,0,0,1,0,1,0,1,1
Wildfire,62,62,62,62,52,0,62,62,62,62,62,32,22,26,40,54,17,42,62


In [39]:
#Reading coordinates csv file and creating new dataframe
coordinates_location_df = pd.read_csv(coordinates_csv)
clean_coordinates_index = coordinates_location_df.index
coordinates_location_df = coordinates_location_df.loc[:, ["Location", "Dis No", "Lat", "Long"]]
#coordinates_location_df

In [57]:
#Northwest coordinates >40N, >110W, and Hawaii
#I need to first create a dataframe based on Longitude, <-130W, >-110W
fourth_quintant_df = coordinates_location_df.loc[(coordinates_location_df["Long"] > -130) & (coordinates_location_df["Long"] < -110), :]
#Create dataframe based on Latitude conditions >40N
fourth_quintant_n_df = fourth_quintant_df.loc[fourth_quintant_df["Lat"] > 40, :].reset_index(drop=True)
#Create dataframe based on Latitude conditions <50N
fourth_quintant_s_df = fourth_quintant_df.loc[fourth_quintant_df["Lat"] < 50, :].reset_index(drop=True)
#Create dataframes based on unique locations within the set conditions for the NW
affected_area_n_df = len(fourth_quintant_n_df["Location"].unique())
affected_area_s_df = len(fourth_quintant_s_df["Location"].unique())

In [58]:
#affected_area_n_df
#print(fourth_quintant_n_df)

In [59]:
#affected_area_s_df

In [101]:
#I need to merge lat dataframes along Dis No
combined_quintant_lat_df = pd.merge(fourth_quintant_n_df, fourth_quintant_s_df, on = "Dis No", how="inner")
#combined_quintant_lat_df
#I need to merge long dataframe with new combined lat dataframe
combined_quintant_nw_df = pd.merge(combined_quintant_lat_df, fourth_quintant_df, on = "Dis No", how="inner")
combined_quintant_nw_df

,Location_x,Dis No,Lat_x,Long_x,Location_y,Lat_y,Long_y,Location,Lat,Long
0,Richland,2000-0338-USA,46.29,-119.28,Richland,46.29,-119.28,Richland,46.29,-119.28
1,Idaho,2000-0465-USA,44.50,-114.25,Arizona,34.50,-111.50,Arizona,34.50,-111.50
2,Idaho,2000-0465-USA,44.50,-114.25,Arizona,34.50,-111.50,Idaho,44.50,-114.25
3,Idaho,2000-0465-USA,44.50,-114.25,Arizona,34.50,-111.50,Nevada,39.25,-116.75
4,Idaho,2000-0465-USA,44.50,-114.25,Arizona,34.50,-111.50,Oregon,44.00,-120.50
...,...,...,...,...,...,...,...,...,...,...
859,Butte,2017-0434-USA,46.00,-112.53,Nevada,39.25,-116.75,Nevada,39.25,-116.75
860,Shasta,2018-0258-USA,40.77,-122.03,Shasta,40.77,-122.03,Shasta,40.77,-122.03
861,Shasta,2018-0258-USA,40.77,-122.03,Shasta,40.77,-122.03,Colusa,39.21,-122.01
862,Shasta,2018-0258-USA,40.77,-122.03,Colusa,39.21,-122.01,Shasta,40.77,-122.03


In [103]:
#I need to create a new dataframe combining the new combine quintant coordinates and the weather disaster data
combined_data_nw_df = pd.merge(combined_quintant_nw_df, wDisaster_df, on = "Dis No", how="outer")
combined_data_nw_df
#I need to recreate dataframe by dropping all NaN values
combined_data_nw_df.replace({'Location_x': 'NaN'}, {'Location_x': float('NaN')})
combined_data_nw_df.replace({'Total Deaths': 'NaN'}, {'Total Deaths': float('NaN')})
combined_data_nw_df.replace({'Total Injured': 'NaN'}, {'No Injured': float('NaN')})
clean_combined_nw_df = combined_data_nw_df.dropna(subset=["Location_x", "Total Deaths", "No Injured"])
clean_combined_nw_df


,Location_x,Dis No,Lat_x,Long_x,Location_y,Lat_y,Long_y,Location_x,Lat,Long,...,Start Month,End Month,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI
184,Armstrong,2002-0310-USA,46.76,-117.23,Armstrong,46.76,-117.23,Armstrong,46.76,-117.23,...,4,5.0,10.0,100.0,NaN,NaN,100.0,2000500.0,2200000.0,70.357819
185,Sumner,2002-0700-USA,47.20,-122.24,Sumner,47.20,-122.24,Sumner,47.20,-122.24,...,11,11.0,36.0,115.0,3600.0,351.0,4066.0,450500.0,700000.0,70.357819
186,Sumner,2002-0700-USA,47.20,-122.24,Sumner,47.20,-122.24,Morgan,41.04,-111.68,...,11,11.0,36.0,115.0,3600.0,351.0,4066.0,450500.0,700000.0,70.357819
187,Sumner,2002-0700-USA,47.20,-122.24,Sumner,47.20,-122.24,Logan,41.74,-111.83,...,11,11.0,36.0,115.0,3600.0,351.0,4066.0,450500.0,700000.0,70.357819
188,Sumner,2002-0700-USA,47.20,-122.24,Morgan,41.04,-111.68,Sumner,47.20,-122.24,...,11,11.0,36.0,115.0,3600.0,351.0,4066.0,450500.0,700000.0,70.357819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,Butte,2017-0434-USA,46.00,-112.53,Nevada,39.25,-116.75,Napa,38.48,-122.33,...,10,10.0,30.0,185.0,NaN,9000.0,9185.0,9600000.0,13000000.0,95.878166
856,Butte,2017-0434-USA,46.00,-112.53,Nevada,39.25,-116.75,Sonoma,38.29,-122.46,...,10,10.0,30.0,185.0,NaN,9000.0,9185.0,9600000.0,13000000.0,95.878166
857,Butte,2017-0434-USA,46.00,-112.53,Nevada,39.25,-116.75,Solano,32.23,-121.95,...,10,10.0,30.0,185.0,NaN,9000.0,9185.0,9600000.0,13000000.0,95.878166
858,Butte,2017-0434-USA,46.00,-112.53,Nevada,39.25,-116.75,Butte,46.00,-112.53,...,10,10.0,30.0,185.0,NaN,9000.0,9185.0,9600000.0,13000000.0,95.878166


In [73]:
#I need to clean up dataframe where the Location column has no information
#clean_ = combined_data_nw_df.iloc[:,0]
# clean_loc.dropna()
# clean_loc
# clean_data_nw_loc_df = pd.merge(combined_data_nw_df, clean_loc)
# clean_data_nw_loc_df

In [106]:
#Creating a trimmed DF of dropped duplicates for more accurate calculations
unique_dis_nw_df = clean_combined_nw_df.drop_duplicates(subset="Dis No", keep="first")
uniqueThirdQuintantNorth = unique_dis_nw_df.loc[unique_dis_nw_df["Lat"] > 40, :].reset_index(drop=True)
uniqueThirdQuintantSouth = unique_dis_nw_df.loc[unique_dis_nw_df["Lat"] < 40, :].reset_index(drop=True)
uniqueNorthAreasAffected = len(uniqueThirdQuintantNorth["Location_x"].unique())
uniqueSouthAreasAffected = len(uniqueThirdQuintantNorth["Location_x"].unique())
uniqueThirdQuintantNorth

AttributeError: 'DataFrame' object has no attribute 'unique'

In [ ]:
# fourth_quintant = combined_data_df.loc[(coordinates_location_df["Long"] < -90) & (combined_data_df["Long"] > -110), :]
# fourth_quintant_n = fourth_quintant.loc[fourth_quintant["Lat"] > 40, :].reset_index(drop=True)
# fourth_quintant_s = fourth_quintant.loc[fourth_quintant["Lat"] < 40, :].reset_index(drop=True)
# affected_area_n = len(fourth_quintant_n["Location_x"].unique())
# affected_area_s = len(fourth_quintant_s["Location_x"].unique())

In [38]:
#Create df with dropped duplicates types
#dis_no_unique_df = fourth_quintant
disaster_type = combined_data_nw_df["Disaster Type"].unique()
# disaster_type_count_n = fourth_quintant_n["Disaster Type"].value_counts()
# disaster_type_count_s = fourth_quintant_s["Disaster Type"].value_counts()
disaster_type

array(['Storm', 'Wildfire', 'Flood', 'Extreme temperature', 'Drought',
       'Epidemic', 'Earthquake', 'Landslide', 'Volcanic activity'],
      dtype=object)

In [2]:
fourth_quintant_n.loc[fourth_quintant_n["Disaster Type"] == "Storm", :]

NameError: name 'fourth_quintant_n' is not defined

In [3]:
#coordinates_location_df["Location"].value_counts()

In [58]:
# coordinates_data = {
#     "Location": [],
#     "Dis No": [],
#     "Lat": [],
#     "Long": []
# }

# for index, row in coordinates_location_df.iterrows():
#     #print(type(coordinates_data["Location"]))
#     if type(row["Location"]) is str:
#         for location in row["Location"].split(","):
#             newData["Location"].append(location)
#             newData["Dis No"].append(row["Dis No"])
# split_coordinates_df = pd.DataFrame(coordinates_data)
# split_coordinates_df

SyntaxError: invalid syntax (<ipython-input-59-3abbedc0f56d>, line 1)